Code to run the inter-subject RSA analysis on the distance matrices in brain (based on beta maps) and behavior (based on computational model). This analysis is run per brain parcel, and these jobs are parallellized on the computing cluster.

In [2]:
%matplotlib inline
import os, glob, warnings, sys
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

import matplotlib, scipy
import numpy as np
import pandas as pd
import scipy.stats as ss
import matplotlib.pyplot as plt
import seaborn as sns
from nltools.data import Brain_Data, Adjacency
from nltools.stats import one_sample_permutation, two_sample_permutation, correlation_permutation
from nltools.stats import threshold
from nltools.mask import expand_mask, collapse_mask
from scipy.stats import pearsonr,spearmanr
from scipy.spatial.distance import squareform
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity

base_dir = '/project/3014018.02/analysis_mri/DataSharingCollection/'

## Run Intersubject RSA

In [11]:
import subprocess

# Fixed arguments
screen = 'Mult' # 'Multiplier screen' is called 'decision screen' in paper
metric_model = 'euclidean'
metric_brain = 'correlation'
permutation_method = 'vector'
n_permute = 100000
nparcel = 200

# Loops
conds = ['X4']
parcels = range(nparcel)
complementOnly = True # Sometimes jobs go missing when being sent to the cluster => run this cell again to fill in the blanks only

for cond in conds:
    jobids = pd.DataFrame(columns=['cond','parcel','jobid'])
    pathCur = os.path.join(base_dir,
        'Results/3.fMRI-ISRSA/IS-RSA/'+
        'IS-RSA_nparcel-%i_perm-%s_%s%s'%(
        nparcel,permutation_method,screen,cond))
    logPathCur = os.path.join(pathCur,'Logfiles/')
    if os.path.exists(pathCur) == False:
        os.mkdir(pathCur)
    if os.path.exists(logPathCur) == False:
        os.mkdir(logPathCur)
    for parcel in parcels:
        go = True
        if complementOnly:
            if os.path.isfile(os.path.join(pathCur,'parcel%03d.csv'%parcel)):
                go = False
        if go:
            cmd = ['/home/decision/jervbaa/.conda/envs/nltools_env/bin/python2.7',
                '/home/decision/jervbaa/Software/SubmitToCluster.py',
                '-length','0:30:00',
                '-memory','2GB',
                '-name','ISRSA-%s-%03d'%(cond,parcel),
                '-logfiledir',logPathCur,
                '-command','python2.7 %s/Code/5.fMRI-ISRSA/Functions/ComputeISRSA.py'%base_dir +
                   ' %s %s %i %i %s %s %s %i'%(
                    screen,cond,nparcel,parcel,metric_model,metric_brain,
                    permutation_method,n_permute),
                ]
            out = subprocess.check_output(' '.join(cmd),shell=True)
            print out
            jobid = out[-27:-1]
            jobids = jobids.append(pd.DataFrame([[cond,parcel,jobid]],columns=jobids.columns))
    jobids.to_csv(os.path.join(pathCur,'jobids.csv'))